In [19]:
import tensorflow as tf
import datetime
import os
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
import pandas as pd 
import numpy as np



# student shape 22025
student = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/student_15.csv')
book = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/book_15.csv')
book_student = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/dl_book_student_15.csv')
book.head() #book_id / subject/ genre / page /pub year


len(book_student['book_id'].unique()) #133878
len(book['book_id'].unique()) #133878
len(book_student['title'].unique()) #99784

# #look at the student table 
# print(book_student.columns)
book_student = book_student[['student_id' , 'book_id', 'title', 'ddc_short' ,'major']] 
book_student.head() #'student_id' , 'book_id', 'title', 'ddc, major'
book_student.shape #(463358, 5)
book_student.isnull().sum() 

student_id    0
book_id       0
title         0
ddc_short     0
major         0
dtype: int64

## 1 Make book into labels 


In [20]:
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
le.fit(book_student['book_id'])
list(le.classes_)[:10]
len(le.classes_) #133878

import pickle
output = open('Bookid_encoder.pkl', 'wb')
pickle.dump(le, output)
output.close()
le.inverse_transform([83973])

#bookid -> number encoded 

array(['EM0891376'], dtype=object)

In [18]:
#make the student book table 
from tqdm import tqdm
import time

#unique student id 
unique_id = student['student_id'].unique()
# len(unique_id) # students is 22024
df= pd.DataFrame()
readonlyonebook = []

for i in tqdm(unique_id):
  #look at unique id in book_student df 
  student_df = book_student[book_student['student_id'] == i ]
  books = student_df['book_id'].values
  ddcs = student_df['ddc_short'].values  
  major = student_df['major'].values[0]
  

  #change the books into label encoded nums 
  books = le.transform(books).astype(int)

    #remove the last ddc & book 
  ddcs = ddcs[:-1]
  read = books[:-1]
  pred = books[-1]
    
  df =df.append({'student_id' : i, 
             'booklist' : list(read),
             'ddclist' : ddcs,
             'major': major,
             'pred' : int(pred)}, ignore_index=True)


df.head()

100%|██████████| 22024/22024 [18:27<00:00, 19.89it/s]


,booklist,ddclist,major,pred,student_id
0,"[22234, 38591, 38927, 44301, 58994, 59239, 621...","[0, 3, 8, 0, 8, 0, 6, 8, 6, 0, 3, 8, 6, 8, 8, ...",사과대학,109047.0,412737.0
1,"[20164, 27532, 71562, 79470, 79607, 81349, 95259]","[0, 8, 1, 8, 8, 1, 3]",소프트웨어대학,99850.0,572900.0
2,"[1004, 1148, 1716, 4074, 4414, 4807, 5056, 996...","[8, 1, 8, 5, 5, 5, 1, 6, 8, 5, 8, 0, 5, 1, 5, ...",소프트웨어대학,131493.0,991018.0
3,"[41531, 45894, 54247, 56907, 73093, 90789]","[7, 7, 7, 9, 7, 8]",사과대학,128795.0,1424110.0
4,"[8604, 8605, 10238, 22870, 74592, 76788, 84120...","[3, 3, 2, 8, 1, 9, 1, 8, 5, 8, 8, 8]",사과대학,99339.0,1520139.0


In [24]:
df.shape #22024, 5)

(22024, 5)

In [21]:
len(unique_id)

22024

In [25]:
#save student-book-table-input
# original shape : (532791,)

#new shape: 
df#(22025, 4)

df.to_csv('input.csv', index=False)

In [ ]:
# if __name__ == "__main__": 
#   print('load the dataset.............')

#   book = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/1218book.csv')
#   book_student = pd.read_csv('/content/drive/MyDrive/명륜이의 서재/FINAL_DATA/1218_dl_book_student.csv')
  
#   major = book_student[['student_id' , 'major']]
#   book_student = book_student[['student_id' , 'book_id', 'title', 'ddc-short']]
  
  
#   print('start making labels for the books.........................')
#   from sklearn.preprocessing  import LabelEncoder
#   le = LabelEncoder()
#   le.fit(book_student['book_id'])

#   print('make ddc in a simple form.......................')
#   book_student['ddc-short'] = book_student['ddc'].apply(lambda x: ddc_sort(x))
  
#   #make dictionary for book-title 